In [1]:
import matplotlib.pyplot as plt
import aplpy
from astropy.wcs import WCS
from astropy.io import fits, ascii
from astropy.utils.data import get_pkg_data_filename
from astroquery.skyview import SkyView
import ssl
import aplpy
from lens import Lens
from astropy.table import Table
import sqlutil as sqlutil
import numpy as np
import lensutils as util


candidates = Table.read('data/candidatesDistTime.txt',format='ascii')
#candidates = Table.read('data/largpmcandidates.txt',format='ascii')
timeMask = (candidates['Closest_approah_time[JYr]'] > 2018.0) & (candidates['Closest_approach_Dist[mas]'] < 260.0)

#candidates = candidates[timeMask]
TGASlensid = candidates['TGAS_id']
ppmxlid = candidates['ppmxl_id']
#CloseApp = candidates['Closest_approach_Dist[mas]'] 
#timeApp = candidates['Closest_approah_time[JYr]']
#imsize = 2

for i in range(0,len(TGASlensid)):
    
    querystringTGAS = 'select ra, dec, source_id,pmra,pmdec,ref_epoch, phot_g_mean_mag from gaia_dr1.tgas_source where source_id=' + str(TGASlensid[i])
    querystringPPMXL ='select ra, dec, id, epochra, epochdec,jmag,hmag,kmag,b1mag,b2mag,r1mag,r2mag from ppmxl.main where id=' + str(ppmxlid[i])

    lensRa, lensDec, id,pmra,pmdec,ref_epoch, gmag = sqlutil.get(querystringTGAS,
                                                           db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')

    testlens1 = Lens(id[0],lensRa[0],lensDec[0],pmra[0],pmdec[0],ref_epoch[0])

    sourceRa, sourceDec,sourceId, epochra, epochdec, jmag,hmag,kmag,b1mag,b2mag,r1mag,r2mag = sqlutil.get(querystringPPMXL,
                       db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')

    util.plt_lens_env(testlens1,sourceRa[0],sourceDec[0],sourceId[0],[jmag[0],hmag[0],kmag[0],b1mag[0],b2mag[0],r1mag[0],r2mag[0]],gmag[0])
    
    
    


INFO: Auto-setting vmin to  9.248e+03 [aplpy.core]
INFO: Auto-setting vmax to  3.049e+04 [aplpy.core]
INFO: Auto-setting vmin to  4.594e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.595e+04 [aplpy.core]
INFO: Auto-setting vmin to  2.001e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.450e+04 [aplpy.core]
INFO: Auto-setting vmin to  1.946e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.719e+04 [aplpy.core]
INFO: Auto-setting vmin to  1.915e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.422e+04 [aplpy.core]
INFO: Auto-setting vmin to  1.420e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.392e+04 [aplpy.core]
INFO: Auto-setting vmin to  1.880e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.684e+04 [aplpy.core]
INFO: Auto-setting vmin to  1.726e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.520e+04 [aplpy.core]
INFO: Auto-setting vmin to  2.090e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.521e+04 [aplpy.core]
INFO: Auto-setting vmin to  1.339e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.5

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


INFO: Auto-setting vmin to  3.958e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.306e+04 [aplpy.core]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


INFO: Auto-setting vmin to  2.080e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.302e+04 [aplpy.core]


...

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


INFO: Auto-setting vmin to  2.066e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.366e+04 [aplpy.core]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


INFO: Auto-setting vmin to  2.330e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.351e+04 [aplpy.core]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
